# Transformer
*Transformer network with Multi-Head Attention*

In [1]:
from chatbot.transformer import Transformer
# import pandas as pd

In [2]:
# df = pd.read_csv('../data/translation/dataset.csv', sep=';')
tf = Transformer(lang='en-us', num_epoch=100)

In [3]:
tf.fit(path='data/conv-ai-2017')

Data loaded with hyperparams: Corpus(lang='en-us', max_length=15, batch_size=64, buffer_size=20000, _start_token=[10619], _end_token=[10620], _vocab_size=10621).
<keras.engine.functional.Functional object at 0x000001A1EF1387C0> compiled successfully.
Epoch 1/100


InvalidArgumentError: in user code:

    File "c:\ProgramData\miniforge3\envs\tfenv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "d:\Users\zer0deck\Documents\Work\python\SW\transformer_chatbot\chatbot\transformer.py", line 181, in _count_f1  *
        with tf.compat.v1.Session() as sess:  print(y_pred.eval())

    InvalidArgumentError: Graph execution error:
    
    Detected at node 'transformer/decoder/decoder_layer_0/attention_1/dense_36/Tensordot/ReadVariableOp/resource' defined at (most recent call last):
        File "c:\ProgramData\miniforge3\envs\tfenv\lib\runpy.py", line 196, in _run_module_as_main
          return _run_code(code, main_globals, None,
        File "c:\ProgramData\miniforge3\envs\tfenv\lib\runpy.py", line 86, in _run_code
          exec(code, run_globals)
        File "c:\ProgramData\miniforge3\envs\tfenv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
          

In [ ]:
tf.save_to_folder(path='trained_models/transformer/conv-ai-2017')

# DNN

*Bag-Of-Words based DNN network*

In [ ]:
# import nltk
# from nltk.stem.lancaster import LancasterStemmer
# import numpy as np
# import tflearn
# import tensorflow as tf
# import json
# import pickle
# import random

# nltk.download("punkt")

# #Loading intents.json
# with open('intents.json') as intents:
#   data = json.load(intents)

# stemmer = LancasterStemmer()

# try:
#     with open('data.pickle','rb') as f:
#         words, labels, training, output = pickle.load(f)
# except:
# # Fetching and Feeding information--
#     words = []
#     labels = []
#     x_docs = []
#     y_docs = []

#     for intent in data['intents']:
#         for pattern in intent['patterns']:
#             wrds = nltk.word_tokenize(pattern)
#             words.extend(wrds)
#             x_docs.append(wrds)
#             y_docs.append(intent['tag'])

#             if intent['tag'] not in labels:
#                 labels.append(intent['tag'])

#     words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
#     words = sorted(list(set(words)))
#     labels = sorted(labels)

#     training = []
#     output = []

#     out_empty = [0 for _ in range(len(labels))]

#     # One hot encoding, Converting the words to numerals
#     for x, doc in enumerate(x_docs):
#         bag = []
#         wrds = [stemmer.stem(w) for w in doc]
#         for w in words:
#             if w in wrds:
#                 bag.append(1)
#             else:
#                 bag.append(0)


#         output_row = out_empty[:]
#         output_row[labels.index(y_docs[x])] = 1

#         training.append(bag)
#         output.append(output_row)


#     training = np.array(training)
#     output = np.array(output)

#     with open('data.pickle','wb') as f:
#         pickle.dump((words, labels, training, output), f)


# net = tflearn.input_data(shape=[None, len(training[0])])
# net = tflearn.fully_connected(net, 8)
# net = tflearn.fully_connected(net, 8)
# net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
# net = tflearn.regression(net)

# model = tflearn.DNN(net)

# try:
#     model.load("model.tflearn")
# except:

#     model.fit(training, output, n_epoch=100, batch_size=8, show_metric=True)
#     model.save('model.tflearn')


# def bag_of_words(s, words):
#     bag = [0 for _ in range(len(words))]
#     s_words = nltk.word_tokenize(s)
#     s_words = [stemmer.stem(word.lower()) for word in s_words]

#     for se in s_words:
#         for i, w in enumerate(words):
#             if w == se:
#                 bag[i] = 1

#     return np.array(bag)


# def chat():
#     print("The bot is ready to talk!!(Type 'quit' to exit)")
#     while True:
#         inp = input("\nYou: ")
#         if inp.lower() == 'quit':
#             break

#     #Porbability of correct response 
#         results = model.predict([bag_of_words(inp, words)])

#     # Picking the greatest number from probability
#         results_index = np.argmax(results)

#         tag = labels[results_index]


#         for tg in data['intents']:

#             if tg['tag'] == tag:
#                 responses = tg['responses']
#             print("Bot:" + random.choice(responses))


# chat()

# Default Sequential model